# Approximating the surprisal of adverbs

<strong>Author:</strong> Alison Y. Kim


### Definition
<strong>Surprisal</strong> (a.k.a. information content) is a measure of the amount of information gained when an event occurs which had some probability value associated with it. Mathematically, it can be represented as such: for some instance or outcome $ x_i $ of random variable $ X $, which can take on values $ x_1, x_2, ... $, and the probability of outcome $ x_{i} $, $ p(x_{i}) $, the surprisal of $ x_{i} $ is given by $$ h(x_{i}) = -\log_{2}{p(x_{i})} \text{ bits} $$
* $ p(x_{i}) = 1 \Rightarrow h(x_{i}) = 0 \text{ bits} $
* $ p(x_{i}) = 0 \Rightarrow h(x_{i}) = \infty \text{ bits} $

In our experiment, one of the values of interest is the surprisal of an adverb.

### Methods

To approximate token probabilities, we use <a href="https://huggingface.co/gpt2-large">GPT-2 large</a>, an English language model (LM). It is accessible via the Hugging Face framework.<br>

For tensor manipulation and operations, we use PyTorch.

### Calculation: Sentential semantic complexity

#### Import statements

In [2]:
from typing import List, Tuple, Union

import nltk
nltk.download('punkt')
import torch
from transformers import PreTrainedTokenizerFast, GPT2TokenizerFast, GPT2LMHeadModel


lm = GPT2LMHeadModel.from_pretrained('gpt2-large')
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2-large')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alisonykim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Define sentences

Parallel corpora (```common``` = sentences with commonly appearing adverbs, ```rare``` = sentences with rarely appearing adverbs)

In [3]:
common = [
	'They looked at each other happily.',
	'The experienced doctor performed the operation sucessfully.',
	'She did not move, continuing to stare at me passionately.',
	'The dogs barked unexpectedly.',
	'The man on the boat waved angrily.',
	'One important person has been continuously absent.',
	'By the swimming pool, the neighbour waited nervously.',
	'It is so easy to be occasionally charitable.',
	'She clings to her marriage desperately.',
	'In this country, racism is spreading constantly.',
	'The little girl screamed and stamped her foot emotionally.',
	'The guests avoided all political discussion carefully.',
	'The day started normally.',
	'She grabbed the microphone stepped onto the stage confidently.',
	'They have been misleading you most unacceptably.',
	'To meet the deadline, the team worked efficiently.',
	'The student studied for the exam carefully.'
]

In [4]:
rare = [
	'They looked at each other amiably.',
	'The experienced doctor performed the operation dexterously.',
	'She did not move, continuing to stare at me belligerently.',
	'The dogs barked ferociously.',
	'The man on the boat waved affably.',
	'One important person has been conspicuously absent.',
	'By the swimming pool, the neighbour waited languidly.',
	'It is so easy to be vicariously charitable.',
	'She clings to her marriage tenaciously.',
	'In this country, racism has been spreading insidiously.',
	'The little girl screamed and stamped her foot petulantly.',
	'The guests avoided all political discussion sedulously.',
	'The day started mundanely.',
	'She grabbed the microphone and stepped onto the stage audaciously.',
	'They have been misleading you most egregiously.',
	'To meet the deadline, the team worked assiduously.',
	'The student studied for the exam sedulously.'
]

### Calculating adverb surprisal
* <strong>Step 1:</strong> Tokenize sentences, i.e. map each token to an index in the LM's vocabulary.
	* <strong>Step 1.5:</strong> Extract the adverb from the sentence and tokenize it separately from the entire sentence. (Some words like "sedulously" are mapped to more than 1 item in GPT-2's vocabulary. Therefore, we must tokenize the adverb separately from the entire sentence. See: <strong>Note</strong>.)
* <strong>Step 2:</strong> Add \<BOS\> and <\EOS\> tokens to the sequence so that the probabilities we get are properly conditioned on the preceding tokens.
* <strong>Step 3:</strong> Calculate tokens' conditional logits (pre-softmax probabilities) under the LM.
* <strong>Step 4:</strong> Shift the label (token IDs) and logits tensors.
	* <strong>Logic:</strong> Here, we are interested in the <em>conditional</em> probabilities of words. Therefore, the conditional probability of \<BOS\> given previous tokens does not make sense to consider (there are no tokens preceding \<BOS\> in the sentence). Similarly, the probabilities over time steps after \<BOS\> are not of interest. The labels and logits tensors are shifted accordingly.
* <strong>Step 5:</strong> Calculate the negative log likelihood of the vocabulary item(s) corresponding to the adverb.
* <strong>Step 6:</strong> Extract the logits corresponding to the token position(s) of the adverb.

<strong>Note:</strong> Because some adverbs are mapped to more than 1 vocabulary item, the helper function below is designed to identify the vocabulary item(s) corresponding to the adverb.

In [5]:
def identify_adverb_ids(
    sent: str,
    tokenizer: PreTrainedTokenizerFast
) -> Union[Tuple[str, List[int], List[int]], Tuple[str, None]]:
    """
    Identify the vocabulary ID(s) corresponding to the adverb.
    
    Returns:
        Tuple of adverb vocabulary ID(s) and adverb token position(s)
    """
    sent_tokenized = nltk.word_tokenize(sent, language='english')
    # Adverbs in this experiment end with 'ly'
    adverb = next((token for token in sent_tokenized if token.endswith('ly')))
    sent_ids = [tokenizer.bos_token_id] + tokenizer(sent)['input_ids'] + [tokenizer.eos_token_id]
    # Reverse-engineer the encoding (via, surprise^, decoding) to find corresponding ID(s)
    # ^No pun intended
    for i in range(len(sent_ids)-1):
        curr_id_decoded = tokenizer.decode(sent_ids[i]).strip()
        next_id_decoded = tokenizer.decode(sent_ids[i+1]).strip()
        if curr_id_decoded == adverb:
            return adverb, [sent_ids[i]], [i]
        elif curr_id_decoded + next_id_decoded == adverb:
            return adverb, sent_ids[i:i+2], [i, i+1]
        elif len(curr_id_decoded + next_id_decoded) < len(adverb) and curr_id_decoded + next_id_decoded in adverb: # Substring in ```adverb``` but not equal
            try:
                next_next_id_decoded = tokenizer.decode(sent_ids[i+2]).strip()
                if curr_id_decoded + next_id_decoded + next_next_id_decoded == adverb:
                    return adverb, sent_ids[i:i+3], [i, i+1, i+2]
            except IndexError:
                return adverb, sent_ids[i:i+2], [i, i+1]
    return adverb, None

##### Common adverb sentences

In [6]:
adverb_surprisal_common = dict()
for sent in common:
	# Step 1
	sent_tokenized = tokenizer(sent)['input_ids']
	# Step 1.5
	adverb, adverb_tokenized, adverb_time_step = identify_adverb_ids(sent, tokenizer)
	# Step 2
	input_ids = torch.tensor([tokenizer.bos_token_id] + sent_tokenized + [tokenizer.eos_token_id])
	# Step 3
	with torch.no_grad():
		outputs = lm(input_ids, labels=input_ids)
	# Step 4
	labels_shifted = input_ids[..., 1:].contiguous()
	adverb_time_step_shifted = [time_step + 1 for time_step in adverb_time_step] # Shift time steps
	logits = outputs['logits']
	logits_shifted = logits[..., :-1, :].contiguous()
	assert logits_shifted.size(0) == labels_shifted.size(0) # As many labels as logits
	# Step 5
	nlls = -1 * torch.log_softmax(logits, dim=-1) # Token surprisal
	# Step 6
	adverb_surprisal_common[adverb] = [
		nlls[time_step][adverb_id].item()
		for time_step, adverb_id in zip(adverb_time_step_shifted, adverb_tokenized)
	]

In [7]:
adverb_surprisal_common

{'happily': [16.239721298217773],
 'sucessfully': [16.785587310791016, 20.889488220214844, 17.696590423583984],
 'passionately': [17.545166015625],
 'unexpectedly': [15.709169387817383],
 'angrily': [15.555511474609375],
 'absent': [12.684146881103516],
 'nervously': [15.868431091308594],
 'charitable': [11.97054672241211],
 'desperately': [13.908486366271973],
 'constantly': [15.275320053100586],
 'emotionally': [14.575103759765625],
 'carefully': [14.272598266601562],
 'normally': [15.229681015014648],
 'confidently': [16.01131820678711],
 'unacceptably': [18.11367416381836, 20.517274856567383, 18.268041610717773],
 'efficiently': [16.871641159057617]}

##### Rare adverb sentences

In [8]:
adverb_surprisal_rare = dict()
for sent in rare:
	# Step 1
	sent_tokenized = tokenizer(sent)['input_ids']
	# Step 1.5
	adverb, adverb_tokenized, adverb_time_step = identify_adverb_ids(sent, tokenizer)
	# Step 2
	input_ids = torch.tensor([tokenizer.bos_token_id] + sent_tokenized + [tokenizer.eos_token_id])
	# Step 3
	with torch.no_grad():
		outputs = lm(input_ids, labels=input_ids)
	# Step 4
	labels_shifted = input_ids[..., 1:].contiguous()
	adverb_time_step_shifted = [time_step + 1 for time_step in adverb_time_step] # Shift time steps
	logits = outputs['logits']
	logits_shifted = logits[..., :-1, :].contiguous()
	assert logits_shifted.size(0) == labels_shifted.size(0) # As many labels as logits
	# Step 5
	nlls = -1 * torch.log_softmax(logits, dim=-1) # Token surprisal
	# Step 6
	adverb_surprisal_rare[adverb] = [
		nlls[time_step][adverb_id].item()
		for time_step, adverb_id in zip(adverb_time_step_shifted, adverb_tokenized)
	]

In [9]:
adverb_surprisal_rare

{'amiably': [12.872679710388184, 21.171295166015625],
 'dexterously': [15.868247985839844, 17.799373626708984],
 'belligerently': [18.694435119628906, 22.017610549926758, 14.72154426574707],
 'ferociously': [24.02274513244629, 16.825103759765625, 18.945785522460938],
 'affably': [14.149466514587402, 19.266386032104492],
 'absent': [13.07290267944336],
 'languidly': [15.334259986877441, 16.413345336914062, 14.369363784790039],
 'charitable': [12.827916145324707],
 'tenaciously': [10.272384643554688, 21.092113494873047],
 'insidiously': [21.785551071166992, 17.391551971435547, 19.25149154663086],
 'petulantly': [24.11342430114746, 17.563024520874023, 19.76900863647461],
 'sedulously': [12.262739181518555, 19.376073837280273],
 'mundanely': [17.870346069335938, 13.98317813873291],
 'audaciously': [12.411210060119629, 21.852031707763672],
 'egregiously': [15.909463882446289, 18.83255386352539],
 'assiduously': [24.112695693969727, 17.81847381591797, 21.968400955200195]}